In [0]:
import tensorflow as tf
import numpy as np
import keras
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    8192/11490434 [..............................] - ETA: 0s

Using TensorFlow backend.


 4202496/11490434 [=========>....................] - ETA: 0s

In [0]:
x_test.shape

In [0]:
# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

In [0]:
# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, LeakyReLU, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

# Creating a Sequential Model and adding the layers
model = Sequential()
model.add(Conv2D(16, kernel_size=(3,3)))
model.add(Conv2D(16, kernel_size=(3,3), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3,3)))
model.add(Conv2D(32, kernel_size=(5,5)))
model.add(Conv2D(64, kernel_size=(1,1)))
model.add(Conv2D(128, kernel_size=(1,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model.add(Dense(128, activation='relu', kernel_initializer="he_normal"))
model.add(BatchNormalization())
# model.add(Dropout(0.05))
model.add(Dense(64, activation='relu', kernel_initializer="he_normal"))
model.add(BatchNormalization())
# model.add(Dropout(0.05))
model.add(Dense(32, activation='relu', kernel_initializer="he_normal"))
model.add(BatchNormalization())
model.add(Dropout(0.3))


model.add(Dense(10,activation=tf.nn.softmax))

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
history = model.fit(x=x_train,y=y_train, validation_split=0.1, epochs=20, batch_size=64, callbacks = [ModelCheckpoint('/content/model.h5',save_best_only=True)])

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [0]:
y_pred=model.predict(x_test)
y_pred=np.argmax(y_pred, axis=1)

In [0]:
con=tf.math.confusion_matrix(y_test,y_pred)
sess = tf.Session()
with sess.as_default():
        print(sess.run(con))

In [0]:
import matplotlib.pyplot as plt


In [0]:
from sklearn.metrics import confusion_matrix
conf_mx = confusion_matrix(y_test, y_pred)

In [0]:
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()

In [0]:
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums

In [0]:
np.fill_diagonal(norm_conf_mx, 0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.show()

In [0]:
model = load_model('/content/model.h5')
result = model.evaluate(x_test, y_test)
result

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(15, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [0]:
model.save